# Final Project Notebook
## Importing/Installing Relevant Libraries

In [1]:

!pip install bs4
!pip install geopy
!pip install folium
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import geopy as gepoy
from geopy.geocoders import Nominatim
import folium as folium
from sklearn.cluster import KMeans
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

     |████████████████████████████████| 115 kB 12.1 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=a56c9e604367fd3064968cf0b5d18a29da2d368dea2866e03ce32b42742f3a61
  Stored in directory: /home/jovyan/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
     |████████████████████████████████| 118 kB 12.9 MB/s eta 0:00:0101
     |████████████████████████████████| 94 kB 2.4 MB/s  eta 0:00:01


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method

## Setting up Beautiful Soup

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')

## Scraping Table

In [ ]:
able_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df

## Adding Longitude and Latitude to DataFrame

In [ ]:

url2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
df2 = pd.read_csv(url2)
df = pd.merge(df, df2, on='Postal Code', how='inner')
df

## Performing KMeans Clustering on Data

In [ ]:
kclusters = 5

coords = df[["Latitude", "Longitude"]]

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(coords)
df.insert(0, 'Cluster Labels', kmeans.labels_)

## Visualizing Clusters

In [ ]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print((latitude, longitude))

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Getting Venue Information from FourSquare

In [ ]:
CLIENT_ID = 'N3E5BJBASSRHGZPLTROGRXCKCKAHGH034JN4K4U3A5DOXEOW' # your Foursquare ID
CLIENT_SECRET = 'GWYLLB14V2R12OPVAF3V1VMXUEAUNHGYC0KO23E3SVXLUFKW' # your Foursquare Secret
ACCESS_TOKEN = '1NLIGRDKZGJZBBMUQFAA15CADMRFAXTKPLBELZUPA52Q4NF5' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

venues_df = pd.DataFrame()
radius = 322 #approximately 2 miles
index_1=0
index_2=0
i=0
while i < df.shape[0]:
    latitude = df["Latitude"][i]
    longitude = df["Longitude"][i]
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    while(index_2<len(results['response']['groups'][0]['items'])):
        temp_df = json_normalize(results['response']['groups'][0]['items'][index_2]['venue'])

        venues_df = venues_df.append(temp_df)
        index_2+=1
    index_2=0
    index_1+=1
    venues_df.drop_duplicates(subset ="id",keep = False, inplace = True)
venues_df


## Visualizing Venues Clustering

In [ ]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print((latitude, longitude))

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(venues_df['location.lat'], venues_df['location.lng'], venues_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters